In [1]:
'''
#itemid 
1 = shorts
2 = caps
3 = T-shirt
4 = sandals
5 = sweater
6 = cardigan
7 = raincoat
8 = coat
9 = gloves
10 = jeans
11 = sneakers
12 = dress
13 = suit
14 = pants
15 = vest
16 = blouse
17 = jacket
18 = sunglasses
19 = bag
20 = shoes
'''

'\n#itemid \n1 = shorts\n2 = caps\n3 = T-shirt\n4 = sandals\n5 = sweater\n6 = cardigan\n7 = raincoat\n8 = coat\n9 = gloves\n10 = jeans\n11 = sneakers\n12 = dress\n13 = suit\n14 = pants\n15 = vest\n16 = blouse\n17 = jacket\n18 = sunglasses\n19 = bag\n20 = shoes\n'

In [2]:
# package import
from google_images_download import google_images_download 
from imageai.Prediction import ImagePrediction
from surprise import Reader, Dataset, SVD, evaluate
from PIL import Image
from io import BytesIO
import pandas as pd 
import csv
import random
import requests
import warnings

warnings.filterwarnings(action='ignore') 

# 아이템 키워드 리스트 작성
item_keyword = ['shorts','caps','T-shirt','sandals','sweater',
                'cardigan','raincoat','coat','gloves','jeans',
                'sneakers','dress','suit','pants','vest',
                'blouse','jacket','sunglasses','bag','shoes']
# 사용자-아이템 평가 csv 파일 
ratings = pd.read_csv('item_rating.csv')


C:\Users\User\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
def login():
    global gender
    
    # 성별입력, 새로운 유저 아이디 생성
    gender = input('성별을 입력해주세요: ')
    new_userId = int(ratings.loc[len(ratings.index) - 1]['userId']) + 1
    print("userId: " + str(new_userId) + "\n성별: " +gender)
    
    #rand_recommend(gender,new_userId)  

In [4]:
def rand_recommend(gender,new_userId):
    global ratings
    
    # 반복문으로 원하는 만큼 아이템을 추천 받는다.
    r_num = int(input('추천 받을 횟수를 입력 하세요: '))
    for a in range(1,r_num+1):
        # 아이템 리스트중에서 랜덤하게 하나를 추천 
        item_Id = random.randint(1,20)
        keywords = str("20대 " + gender + " " + item_keyword[(item_Id)-1] + " 추천")
        response = google_images_download.googleimagesdownload()
        arguments = {"keywords":keywords,"limit":1,"output_directory":'image',"no_download":True, "safe_search":True}
        paths = response.download(arguments)
        
        # 추천받은 아이템에 대한 점수 평가 
        while True:
            itemscore = int(input('점수를 평가해주세요(1~5점): '))
            if 1 <= itemscore and itemscore <= 5:
                break
            else:
                print('warning: 1점에서 5점사이의 값을 입력하세요')
        if gender == '남자':
            new_gender = 1
        else: new_gender = 2 
        new_itemId = item_Id
        new_rating = itemscore
        
        # 평가한 점수를 기존 csv파일에 추가한다.
        ratings.loc[len(ratings.index)] = [new_userId, new_gender, new_itemId, new_rating]
        ratings = ratings.set_index("userId")
        ratings.to_csv('item_rating.csv')
        ratings = pd.read_csv('item_rating.csv')
        
    #svd_ratings(ratings,new_userId)

In [5]:
def svd_ratings(ratings,new_userId):
    reader = Reader()
    svd = SVD()
    
    # 데이터셋 학습시키기 
    data = Dataset.load_from_df(ratings[['userId', 'itemId', 'rating']], reader)
    trainset = data.build_full_trainset() 
    svd.fit(trainset)
    
    # 사용자가 평가를 안내린 아이템들 중 무작위로 3개의 점수를 예측한다. 
    lst_itemId = list(range(1,21))
    lst_ratings = list(ratings[ratings.userId == new_userId]['itemId'].values)
    notrating_itemId =list(set(lst_itemId)-set(lst_ratings))
    rand_itemId = random.sample(notrating_itemId,3)
    a = 0
    svd_rating = []
    
    # 평가를 안내린 아이템, 그 아이템의 예측 점수를 각각 key와 value로 dic생성 
    for i in range(1,4): 
        svd_rating.append(round((svd.predict(new_userId,rand_itemId[a]).est), 1))
        a += 1
    rating_dic = dict(zip(rand_itemId,svd_rating))
    
    # 3개의 항목중에 가장 높은 에측 값을 가진 아이템의 코드를 추천한다 
    recommend_itemId = max(rating_dic, key=rating_dic.get)
    
    #final_recommend(recommend_itemId)

In [6]:
def final_recommend(recommend_itemId):
    
    # 추천 받은 코드로 키워드를 생성하여 아이템을 추천한다.
    keywords = str("20대 " + gender + " " + item_keyword[(recommend_itemId)-1] + " 추천")                      
    response = google_images_download.googleimagesdownload()
    arguments = {"keywords":keywords,"limit":1,"output_directory":'image',"no_download":True, "safe_search":True}
    
    print('사용자에게 추천는 아이템입니다')
    paths = response.download(arguments)
    
    # 추천받은 아이템의 url주소를 파싱하여 img변수에 바인딩 
    '''
    url = paths[keywords][0]
    resp = requests.get(url)
    img = Image.open(BytesIO(resp.content))            
   
    '''

In [3]:
# 전체 코드

from google_images_download import google_images_download 
from imageai.Prediction import ImagePrediction
from surprise import Reader, Dataset, SVD, evaluate
from PIL import Image
from io import BytesIO
import pandas as pd 
import csv
import random
import requests
import warnings

warnings.filterwarnings(action='ignore') 


item_keyword = ['shorts','caps','T-shirt','sandals','sweater',
                'cardigan','raincoat','coat','gloves','jeans',
                'sneakers','dress','suit','pants','vest',
                'blouse','jacket','sunglasses','bag','shoes']

ratings = pd.read_csv('item_rating.csv')


def login():
    global gender
    
    gender = input('성별을 입력해주세요: ')
    new_userId = int(ratings.loc[len(ratings.index) - 1]['userId']) + 1
    print("userId: " + str(new_userId) + "\n성별: " +gender)
    rand_recommend(gender,new_userId)    

def rand_recommend(gender,new_userId):
    global ratings
    r_num = int(input('추천 받을 횟수를 입력 하세요: '))
    for a in range(1,r_num+1):
        item_Id = random.randint(1,20)
        keywords = str("20대 " + gender + " " + item_keyword[(item_Id)-1] + " 추천")
        response = google_images_download.googleimagesdownload()
        arguments = {"keywords":keywords,"limit":1,"output_directory":'image',"no_download":True, "safe_search":True}
        paths = response.download(arguments)
        
        while True:
            itemscore = int(input('점수를 평가해주세요(1~5점): '))
            if 1 <= itemscore and itemscore <= 5:
                break
            else:
                print('warning: 1점에서 5점사이의 값을 입력하세요')
        if gender == '남자':
            new_gender = 1
        else: new_gender = 2 
        new_itemId = item_Id
        new_rating = itemscore
        
        ratings.loc[len(ratings.index)] = [new_userId, new_gender, new_itemId, new_rating]
        ratings = ratings.set_index("userId")
        ratings.to_csv('item_rating.csv')
        ratings = pd.read_csv('item_rating.csv')
    svd_ratings(ratings,new_userId)
    

def svd_ratings(ratings,new_userId):
    reader = Reader()
    svd = SVD()

    data = Dataset.load_from_df(ratings[['userId', 'itemId', 'rating']], reader)
    trainset = data.build_full_trainset()
     
    svd.fit(trainset)
    lst_itemId = list(range(1,21))
    lst_ratings = list(ratings[ratings.userId == new_userId]['itemId'].values)
    notrating_itemId =list(set(lst_itemId)-set(lst_ratings))
    rand_itemId = random.sample(notrating_itemId,3)
    a = 0
    svd_rating = []
    
    for i in range(1,4): 
        svd_rating.append(round((svd.predict(new_userId,rand_itemId[a]).est), 1))
        a += 1
    
    rating_dic = dict(zip(rand_itemId,svd_rating))
    recommend_itemId = max(rating_dic, key=rating_dic.get)
    final_recommend(recommend_itemId)
    

def final_recommend(recommend_itemId):

    keywords = str("20대 " + gender + " " + item_keyword[(recommend_itemId)-1] + " 추천")                      
    response = google_images_download.googleimagesdownload()
    arguments = {"keywords":keywords,"limit":1,"output_directory":'image',"no_download":True, "safe_search":True}
    
    print('사용자에게 추천하는 아이템입니다')
    paths = response.download(arguments)
    
    '''
    url = paths[keywords][0]
    resp = requests.get(url)
    img = Image.open(BytesIO(resp.content))            
   
    '''
          
    
if __name__ == '__main__':
    login()
    print('프로그램을 종료합니다')
  

성별을 입력해주세요: 여자
userId: 36
성별: 여자
추천 받을 횟수를 입력 하세요: 2

Item no.: 1 --> Item name = 20\ub300 \uc5ec\uc790 T-shirt \ucd94\ucc9c
Evaluating...
Getting URLs without downloading images...
Image URL: http://meidiama.com/web/product/big/201706/423_shop1_842630.gif
Printed url without downloading

Errors: 0

점수를 평가해주세요(1~5점): 1

Item no.: 1 --> Item name = 20\ub300 \uc5ec\uc790 gloves \ucd94\ucc9c
Evaluating...
Getting URLs without downloading images...
Image URL: http://postfiles16.naver.net/MjAxODEyMjlfMTI5/MDAxNTQ2MDQ4NTk1ODE1.pdcqWHRJ9Ss2wIe_Qk8WdR8yTTRuRC9KCY8rAY0Jzq0g.If70h-KIk8xFGpjlficSqC2iXBIN3CZpLQlCpV3sRCAg.JPEG.yesno813/2018-11-28-17-56-13_edit.jpg?type=w1
Printed url without downloading

Errors: 0

점수를 평가해주세요(1~5점): 3
사용자에게 추천는 아이템입니다

Item no.: 1 --> Item name = 20\ub300 \uc5ec\uc790 suit \ucd94\ucc9c
Evaluating...
Getting URLs without downloading images...
Image URL: https://post-phinf.pstatic.net/MjAxODAxMjlfMTMw/MDAxNTE3MjExMzk3NDAy.NSvuR7mvEVyV9QemvGwpI65h5nyffK5G8LHyqom_t3og.